<a href="https://colab.research.google.com/github/jnels4/nfl_predictions/blob/Brit/NFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,121 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-05 03:07:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-09-05 03:07:10 (10.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NFL-Prediction").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://nflpredictionsdataviz.s3.amazonaws.com/spreadspoke_scores.csv"
spark.sparkContext.addFile(url)
NFL_Guru_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("spreadspoke_scores.csv"), sep=",", header=True, inferSchema=True)
NFL_Guru_df.show()

+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|          team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|     9/2/1966|           1966|            1|           false|     Miami Dolphins|        14|        23|     Oakland Raiders|            null|           null

In [6]:
NFL_Guru_df.printSchema()

root
 |-- schedule_date: string (nullable = true)
 |-- schedule_season: integer (nullable = true)
 |-- schedule_week: string (nullable = true)
 |-- schedule_playoff: boolean (nullable = true)
 |-- team_home: string (nullable = true)
 |-- score_home: integer (nullable = true)
 |-- score_away: integer (nullable = true)
 |-- team_away: string (nullable = true)
 |-- team_favorite_id: string (nullable = true)
 |-- spread_favorite: double (nullable = true)
 |-- over_under_line: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- stadium_neutral: boolean (nullable = true)
 |-- weather_temperature: integer (nullable = true)
 |-- weather_wind_mph: integer (nullable = true)
 |-- weather_humidity: integer (nullable = true)
 |-- weather_detail: string (nullable = true)



In [7]:
NFL_guru_current = NFL_Guru_df.where(NFL_Guru_df.schedule_season>1978)
NFL_guru_current.show()

+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|           team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|     9/1/1979|           1979|            1|           false|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|              TB|           

In [8]:
from pyspark.sql import functions as F

NFL_guru_wins = NFL_guru_current.withColumn('home_win',
                                            F.when((F.col('score_home')>F.col('score_away')), 2)\
                                            .when((F.col('score_home')<F.col('score_away')), 1).otherwise(0)                                   
                                            )
NFL_guru_wins.show()

+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+--------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|           team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|home_win|
+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+--------+
|     9/1/1979|           1979|            1|           false|Tampa Bay Buccaneers|        31|        16|       Detroit Lions| 

In [9]:
cols = ('schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff', 'team_favorite_id','stadium_neutral','weather_humidity', 'over_under_line')
NFL_short = NFL_guru_wins.drop(*cols)
NFL_short.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|          null|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|          null|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|          null|       2|
|      Denver Br

In [10]:
NFL_fix_weather = NFL_short.withColumn('weather_detail', F.when(NFL_short.weather_detail=='DOME','1').otherwise('0'))
NFL_fix_weather.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|             0|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|             0|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|             0|       2|
|      Denver Br

In [11]:
NFL_clean = NFL_fix_weather
NFL_clean.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|             0|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|             0|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|             0|       2|
|      Denver Br

In [25]:
NFL_clean = NFL_clean.dropna()
NFL_clean_df = NFL_clean.toPandas()
NFL_clean_df.head()

,team_home,score_home,score_away,team_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,Tampa Bay Buccaneers,31,16,Detroit Lions,-3.0,Houlihan's Stadium,79,9,0,2
1,Buffalo Bills,7,9,Miami Dolphins,-5.0,Ralph Wilson Stadium,74,15,0,1
2,Chicago Bears,6,3,Green Bay Packers,-3.0,Soldier Field,78,11,0,2
3,Denver Broncos,10,0,Cincinnati Bengals,-3.0,Mile High Stadium,69,6,0,2
4,Kansas City Chiefs,14,0,Baltimore Colts,-1.0,Arrowhead Stadium,76,8,0,2


In [26]:
#drop the nas ealier 
NFL_clean_df = NFL_clean_df.dropna()

In [27]:
NFL_guru_jetsbills = NFL_guru_current.filter(((NFL_guru_current.team_home == 'New York Jets') & (NFL_guru_current.team_away == 'Buffalo Bills')) | ((NFL_guru_current.team_home == 'Buffalo Bills') & (NFL_guru_current.team_away == 'New York Jets')))
NFL_guru_jetsbills.show()

+-------------+---------------+-------------+----------------+-------------+----------+----------+-------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|    team_home|score_home|score_away|    team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+-------------+----------+----------+-------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|    9/23/1979|           1979|            4|           false|Buffalo Bills|        46|        31|New York Jets|             BUF|           -3.0|             42|Ralph Wilson Stadium|          fals

In [28]:
NFL_guru_jetsbills = NFL_guru_jetsbills.withColumn('home_win',
                                            F.when((F.col('score_home')>F.col('score_away')), 2)\
                                            .when((F.col('score_home')<F.col('score_away')), 1).otherwise(1)                                   
                                            )

In [29]:
NFL_guru_jetsbills = NFL_guru_jetsbills.drop(*cols)
NFL_guru_jetsbills.show()

+-------------+----------+----------+-------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|    team_home|score_home|score_away|    team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+-------------+----------+----------+-------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Buffalo Bills|        46|        31|New York Jets|           -3.0|Ralph Wilson Stadium|                 50|               9|          null|       2|
|New York Jets|        12|        14|Buffalo Bills|           -6.0|      Giants Stadium|                 52|              10|          null|       1|
|Buffalo Bills|        20|        10|New York Jets|            0.0|Ralph Wilson Stadium|                 68|              13|          null|       2|
|New York Jets|        24|        31|Buffalo Bills|           -4.0|      Giants Stadium|            

In [30]:
NFL_guru_jetsbills = NFL_guru_jetsbills.withColumn('weather_detail', F.when(NFL_short.weather_detail=='DOME','1').otherwise('0'))

In [31]:
NFL_guru_jetsbills_df = NFL_guru_jetsbills.toPandas()
NFL_guru_jetsbills_df.head()

,team_home,score_home,score_away,team_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,Buffalo Bills,46,31,New York Jets,-3.0,Ralph Wilson Stadium,50.0,9.0,0,2
1,New York Jets,12,14,Buffalo Bills,-6.0,Giants Stadium,52.0,10.0,0,1
2,Buffalo Bills,20,10,New York Jets,0.0,Ralph Wilson Stadium,68.0,13.0,0,2
3,New York Jets,24,31,Buffalo Bills,-4.0,Giants Stadium,44.0,12.0,0,1
4,Buffalo Bills,31,0,New York Jets,-4.0,Ralph Wilson Stadium,66.0,8.0,0,2


In [32]:
#drop the nas ealier 
NFL_guru_jetsbills_df =NFL_guru_jetsbills_df.dropna()

In [33]:
def change_string(team_home):
    if team_home == "Buffalo Bills":
        return 1 
    else:
        return 0
    
NFL_guru_jetsbills_df["team_home"]= NFL_guru_jetsbills_df["team_home"].apply(change_string)
NFL_guru_jetsbills_df.head()

,team_home,score_home,score_away,team_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,1,46,31,New York Jets,-3.0,Ralph Wilson Stadium,50.0,9.0,0,2
1,0,12,14,Buffalo Bills,-6.0,Giants Stadium,52.0,10.0,0,1
2,1,20,10,New York Jets,0.0,Ralph Wilson Stadium,68.0,13.0,0,2
3,0,24,31,Buffalo Bills,-4.0,Giants Stadium,44.0,12.0,0,1
4,1,31,0,New York Jets,-4.0,Ralph Wilson Stadium,66.0,8.0,0,2


In [34]:
def change_string(team_away):
    if team_away == "New York Jets":
        return 1 
    else:
        return 0
    
NFL_guru_jetsbills_df["team_away"]= NFL_guru_jetsbills_df["team_away"].apply(change_string)
NFL_guru_jetsbills_df.head()

,team_home,score_home,score_away,team_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,1,46,31,1,-3.0,Ralph Wilson Stadium,50.0,9.0,0,2
1,0,12,14,0,-6.0,Giants Stadium,52.0,10.0,0,1
2,1,20,10,1,0.0,Ralph Wilson Stadium,68.0,13.0,0,2
3,0,24,31,0,-4.0,Giants Stadium,44.0,12.0,0,1
4,1,31,0,1,-4.0,Ralph Wilson Stadium,66.0,8.0,0,2


In [35]:
def change_string(stadium):
    if stadium == "Ralph Wilson Stadium":
        return 2 
    else:
        return 3
    
NFL_guru_jetsbills_df["stadium"]= NFL_guru_jetsbills_df["stadium"].apply(change_string)
NFL_guru_jetsbills_df.head()

,team_home,score_home,score_away,team_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,1,46,31,1,-3.0,2,50.0,9.0,0,2
1,0,12,14,0,-6.0,3,52.0,10.0,0,1
2,1,20,10,1,0.0,2,68.0,13.0,0,2
3,0,24,31,0,-4.0,3,44.0,12.0,0,1
4,1,31,0,1,-4.0,2,66.0,8.0,0,2


for loop to go through the teams- completing the procress automatically rather than going through each match indiv.

In [36]:
results = []
home_teams = NFL_clean.select('team_home').distinct()
home_team_list = home_teams.rdd.map(lambda x: x[0]).collect()
visiting_teams = NFL_clean.select('team_away').distinct()
visiting_team_list = visiting_teams.rdd.map(lambda x: x[0]).collect()
teams_seen = []

In [38]:
import pandas as pd 
save = []
save = pd.DataFrame()
check_df = pd.DataFrame()

In [ ]:
for home_team in home_team_list:
  for visiting_team in visiting_team_list:
    if home_team == visiting_team:
      continue
    else:

      # Subset your dataframe
      temp_df = NFL_clean.filter(((NFL_clean.team_home == home_team) & (NFL_clean.team_away == visiting_team)) | ((NFL_clean.team_home == visiting_team) & (NFL_clean.team_away == home_team)))
        # Dump it to a pandas df
      temp_df_panda = temp_df.toPandas()
      temp_df_panda = temp_df_panda.drop(['team_home', 'team_away'], axis=1)
      check_df = temp_df_panda
      if len(temp_df_panda) > 10:     
        # Get dummies
        stadium_encoded = pd.get_dummies(temp_df_panda, columns=['stadium'])
        save = stadium_encoded
        stadium_encoded.head()
        X = stadium_encoded.drop(columns=['home_win'])
        y = stadium_encoded['home_win']

        # Split into X, Y and training/test sets

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1 )#,stratify = y

        # Fit model
        ros = RandomOverSampler(random_state=1)
        X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
        model = LogisticRegression(solver='lbfgs', random_state=1)
        model.fit(X_resampled, y_resampled)

        # Predict
        y_pred = model.predict(X_test)
        accuracy = balanced_accuracy_score(y_test, y_pred)
        matrix = confusion_matrix(y_test, y_pred)

        #results = pd.DataFrame()
        # Put your results in a dataframe

        results['Team_1'] = home_team
        results['Team_2'] = visiting_team
        results['Accuracy'] = accuracy

        # Store results
        #results.append()
        all_model_results = pd.concat([results])

In [40]:
results = []

home_team_list = df['team_home'].unique()
visiting_team_list = df['team_away'].unique()

teams_seen = []

for home_team, visiting_team in zip(home_team_list, visiting_team_list):

  # Subset your dataframe
  temp_df = NFL_guru_current.filter(((NFL_guru_current.team_home == home_team) & (NFL_guru_current.team_away == visiting_team)) | ((NFL_guru_current.team_home == visiting_team) & (NFL_guru_current.team_away == home_team)))

  # Dump it to a pandas df

  # Get dummies

  # Split into X, Y and training/test sets

  # Fit model

  # Predict
  res = pd.Dataframe()

  # Put your results in a dataframe
  res['Home Team'] = home_team
  res['Visiting Team'] = visiting_team

  # Store results
  results.append()

all_model_results = pd.concat(results)

NameError: ignored

In [41]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [42]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [43]:
NFL_jetsbills_encoded = pd.get_dummies(NFL_guru_jetsbills_df, columns=['stadium'])
NFL_jetsbills_encoded.shape

(79, 11)

In [44]:
#make sure when running the x and y ALL nas need to be removed 
X = pd.get_dummies(NFL_guru_jetsbills_df.drop(columns="home_win"))
y = NFL_guru_jetsbills_df["home_win"]

In [45]:
NFL_jetsbills_encoded = NFL_jetsbills_encoded.dropna()

In [29]:
y.value_counts()

2    42
1    37
Name: home_win, dtype: int64

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
X_train.shape

(59, 10)

In [47]:
from imblearn.ensemble import BalancedRandomForestClassifier
brfc_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [48]:
NFL_jetsbills_encoded = pd.get_dummies(NFL_guru_jetsbills_df, columns=['stadium'])
NFL_jetsbills_encoded.shape

(79, 11)

In [49]:
NFL_jetsbills_encoded = NFL_jetsbills_encoded.dropna()

In [50]:
NFL_jetsbills_encoded.shape

(79, 11)

In [51]:
#interaction between the 2 teams- keeping it seperate from the entire DB cleaned 
X = NFL_jetsbills_encoded.drop(['home_win'], axis=1)
y = NFL_jetsbills_encoded['home_win']


In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)


Random Sam balanced accuracy= 0.

In [53]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({1: 31, 2: 31})

In [54]:
from sklearn.linear_model import LogisticRegression
#convert the team names from strings to float- need to do this earlier with ALL team names- preventative

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [55]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9545454545454546

In [56]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 1, 10]])

In [57]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.90      1.00      0.91      0.95      0.95      0.92         9
          2       1.00      0.91      1.00      0.95      0.95      0.90        11

avg / total       0.96      0.95      0.96      0.95      0.95      0.91        20



SMOTE testing- balanced accuracy 0.955

In [58]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train)
Counter(y_resampled)

Counter({1: 31, 2: 31})

In [59]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [60]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9545454545454546

In [61]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 1, 10]])

In [62]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.90      1.00      0.91      0.95      0.95      0.92         9
          2       1.00      0.91      1.00      0.95      0.95      0.90        11

avg / total       0.96      0.95      0.96      0.95      0.95      0.91        20



SMOTEEN testing- balanced accuracy= 1

In [63]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({1: 29, 2: 30})

In [64]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [65]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [66]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 0, 11]])

In [67]:
print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

          1       1.00      1.00      1.00      1.00      1.00      1.00         9
          2       1.00      1.00      1.00      1.00      1.00      1.00        11

avg / total       1.00      1.00      1.00      1.00      1.00      1.00        20

